# Στοιχεία Φοιτητή

Σκουρτσίδης Γεώργιος 03114307

# Τεχνητή Νοημοσύνη: Εργαστηριακή Άσκηση 2
---

Ο στόχος της εργασίας είναι η κατασκευή ενός συστήματος προτάσεων (Recommendation System) για ταινίες. Οι προτάσεις αυτές θα πηγάζουν τόσο από τα χαρακτηριστικά της ταινίας όσο και από ορισμένες αξιολογήσεις του κάθε χρήστη.

Στα δεδομένα της άσκησης περιλαμβάνονται ένα αρχείο με το όνομα movies_metadata.csv το όποιο περιέχει τα χαρακτηριστικά κάθε ταινίας όπως θέμα, σκηνοθέτης ηθοποιοί, λέξεις κλειδιά κ.α. από το imdb καθώς και τα αρχεία ratings.csv τα όποια περιέχουν πραγματικές αξιολογήσεις χρηστών, χωρισμένες σε train και σε test.


---
## Κατασκευή Περιβάλλοντος Εργασίας
---

### Διάβασμα Αρχείων στο Colab 
Αν η υλοποίηση γίνει στο google colab τότε μπορεί να χρησιμοποιηθεί το google drive ως file system. Για να γίνει Mount το google drive τρέχουμε τον παρακάτω κώδικα και κλικάρουμε στο link που θα μας εμφανιστεί. 

```
from google.colab import drive
drive.mount('/gdrive')
import os
os.listdir('/gdrive/My Drive')
```
Έπειτα στην σελίδα που άνοιξε επιλέγουμε το mail μας και στο επόμενο παράθυρο που θα μας ανοίξει πατάμε Να επιτρέπεται. Στην συνέχεια αντιγράφουμε τον κωδικό που θα μας βγάλει και τον κάνουμε paste στο Input που έχει ανοίξει στο colab. Έτσι πλέον αν έχουμε ανεβάσει ένα αρχείο στο google drive μπορούμε να το βρούμε στην θέση:

```
movies_filename = '/gdrive/My Drive/' + movies_metadata.csv
```
Μπορούμε πλέον κανονικά να δουλέψουμε φτιάχνοντας φακέλους ή αρχεία και γενικότερα κάνοντας οτιδήποτε θα κάναμε αν ήμασταν τοπικά. 




### Prolog μέσω Python
___
Το πακέτο που θα χρησιμοποιηθεί για την επικοινωνία Python και Prolog είναι το pyswip (https://pypi.org/project/pyswip/). Για να δουλέψει το Pyswip  χρειάζεται να υπάρχει το Swi-Prolog το όποιο αν δουλεύουμε τοπικά πρέπει να το εγκαταστήσουμε, ακολουθώντας αντίστοιχες οδηγίες στην σελίδα του εργαλείου. Για να γίνει του Swi-Prolog η εγκατάσταση στο Google Colab πρέπει να τρέξουμε τον παρακάτω κώδικα:

```
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog
```
Σε κάποιο σημείο της εκτέλεσης εμφανίζεται ένα μήνυμα ότι πρέπει να πατήσουμε enter σε ένα input για να συνεχίσει η διαδικασία. Έπειτα από αυτό η εκτέλεση θα συνεχίσει χωρίς κάποιο πρόβλημα.

Τέλος πρέπει να εγκαταστήσουμε το pyswip (**όπου και να δουλεύουμε**) όπως παρακάτω:

```
!pip isntall pyswip
```


---
#Κώδικας για κατασκευή περιβάλλοντος εργασίας
___
##Μόνο για Google Colab

Κώδικας για να γίνει Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
import os
os.listdir('/gdrive/My Drive')
path='/gdrive/My Drive/tn2/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog/stable
!sudo apt-get update
!sudo apt-get install swi-prolog
#install pyswip
!pip install pyswip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.96.24.32.14).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
 Comprehensive Prolog implementation with extensive libraries and development tools.   Primarily targetted at teaching, RDF processing and web-related tasks, such as creating web services or analysing web content.
 More info: https://launchpad.net/~swi-prolog/+archive/ubuntu/stable
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86

# **1o Μέρος: Μελέτη των Metadata, Δημιουργία κόσμου και των βασικών queries.**
---

Στο μέρος 1 και 2 θα ασχοληθείτε και θα εργαστείτε μόνο με το αρχείο movies_metadata.csv.

In [ ]:
import pandas as pd
from pyswip import Prolog

In [ ]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
# Try to increase stack size
Prolog.consult(path +"increase_stack_size.pl")
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv(path + "movies_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,production_companies,production_countries,release_date,gross,duration,spoken_languages,status,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,880674609,148,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,A Plan No One Escapes,Spectre,6.3,4466,2015,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,1084939099,165,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,2012,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,284139100,132,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,2012,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


## Ερώτημα 1α
---
Αρχικά θα πρέπει, αφού μελετήσετε τη δομή και τα χαρακτηριστικά του αρχείου αυτού, να κατασκευάσετε μια βάση γνώσης για την Prolog η όποια ουσιαστικά θα αποτελεί τον κόσμο με τον όποιο θα εργαστείτε στην συνέχεια. Τα κατηγορήματα που θα δημιουργηθούν θα σας βοηθήσουν και στην κατασκευή του recommender και θα είναι της μορφής:

```
director(Movie, Director).
genre(Movie, Genre).
```

In [ ]:
# All labels names 
for col in data.columns: 
    print(col) 

Unnamed: 0
budget
genres
homepage
id
plot_keywords
language
original_title
overview
popularity
production_companies
production_countries
release_date
gross
duration
spoken_languages
status
tagline
movie_title
vote_average
num_voted_users
title_year
country
director_name
actor_1_name
actor_2_name
actor_3_name


In [ ]:
def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

In [ ]:
# Create World
# Ορίζουμε τον κόσμο μας
prolog = Prolog()
prolog.query("make")

# Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
# αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
color = "Colorized"
for row in data.itertuples(index=True, name='Pandas'):
    # Παίρνουμε τα δεδομένα που μας αφορούν
    movie_title = clean_text(getattr(row, 'movie_title'))
    director = clean_text(getattr(row, 'director_name'))
    num_voted_users = clean_text(str(getattr(row, 'num_voted_users')))
    vote_average = clean_text(str(getattr(row, 'vote_average')))
    title_year = clean_text(str(getattr(row, 'title_year')))
    plot_keywords = clean_text(getattr(row, 'plot_keywords'))
    popularity = clean_text(str(getattr(row, 'popularity')))
    duration = clean_text(str(getattr(row, 'duration')))
    
    # Αν δεν είναι null τότε τα αποθηκεύουμε στη λίστα literals
    if movie_title != "UNK":
        if director != "UNK":
            literals.append("director('"+ movie_title +"','" + director + "')")
        if director != "UNK":
            literals.append("popularity('"+ movie_title +"','" + popularity + "')")            
        if num_voted_users != "UNK":
            literals.append("num_voted_users('"+ movie_title +"','" + num_voted_users + "')")            
        if title_year != "UNK":
            literals.append("title_year('"+ movie_title +"','" + title_year + "')")          
        
        # Rating
        if vote_average != "UNK":
          if float(vote_average) > 8.0:
            literals.append("vote('"+ movie_title +"','over_8')")    
          elif float(vote_average) < 8.0 and float(vote_average) > 6.0:
            literals.append("vote('"+ movie_title +"','over_6')")    
          elif float(vote_average) < 6.0 and float(vote_average) > 3.0:
            literals.append("vote('"+ movie_title +"','over_3')")    
          else: # vote_average < 3.0
            literals.append("vote('"+ movie_title +"','under_3')")   

        # Duration (3 categories long,medium short)
        if duration != "UNK":
          if float(duration) > 120:
              literals.append("duration('"+ movie_title +"','long')")    
          elif float(duration) < 120 and float(duration) >60:
              literals.append("duration('"+ movie_title +"','medium')")  
          else:# duration < 60:
              literals.append("duration('"+ movie_title +"','short')")           
        
        # Users Voted (3 categories depending of the number of votes)
        if num_voted_users != "UNK":
          if float(num_voted_users) >8000:
            literals.append("users_voted('"+ movie_title +"','high')")         
          elif float(num_voted_users) < 8000 and float(num_voted_users) > 4000:
            literals.append("users_voted('"+ movie_title +"','medium')")         
          else:
            literals.append("users_voted('"+ movie_title +"','low')")         

        # Keywords and color
        for plot_keyword in plot_keywords.split("|"):
            literals.append("keyword('"+ movie_title +"','"+ plot_keyword +"')")
            if plot_keyword == "black and white":
              color = "Black_and_White"
        literals.append("color('"+ movie_title +"','" + color + "')")
        
        # Genre
        for genre in getattr(row, 'genres').split("|"):
            literals.append("genre('"+ movie_title +"','"+ genre +"')")
        # Actors
        for actor in [getattr(row, 'actor_1_name'), getattr(row, 'actor_2_name'), getattr(row, 'actor_3_name')]:
            literals.append("actors('"+ movie_title +"','"+ actor.replace("'",'"') +"')")
        # Language
        for language in getattr(row, 'language'):
            literals.append("language('"+ movie_title +"','"+ language +"')")
            
# Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
  prolog.assertz(literal)

# Επίσης μπορούμε να κάνουμε consult ένα έτοιμο αρχείο στον κόσμο όπως παρακάτω
#  prolog.consult(path +"db.pl")


## Ερώτημα 1β
---
Αφού δημιουργήσετε τον κόσμο του προβλήματος, στη συνέχεια καλείστε να δημιουργήσετε, σε Prolog, απλούς κανόνες οι οποίοι θα βρίσκουν όλες τις ταινίες με:
1.	Κοινό θέμα (κάποιες λέξεις σχετικά με το genre κοινές)
2.	Αρκετά κοινό θέμα (κάποιες λιγότερες λέξεις σχετικά με το genre κοινές π.χ. 3)
3.	Σχετικά κοινό θέμα (λίγες λέξεις σχετικές με το genre κοινές π.χ. 1)
4.	Κοινός σκηνοθέτης
5.	Ακριβώς ίδια πλοκή (κάποιες λέξεις κλειδιά της πλοκής κοινές)
6.	Σχετικά ίδια πλοκή (κάποιες λιγότερες λέξεις κλειδιά κοινές)
7.	Ίδιους τους βασικούς ηθοποιούς (και τους 3)
8.	Αρκετά ίδιους βασικούς ηθοποιούς (ορισμένους βασικούς ηθοποιούς κοινούς π.χ. 2)
9.	Σχετικά ίδιους ηθοποιούς (π.χ. 1 από τους 3)
10.	Ίδια γλώσσα
11.	Είναι έγχρωμες ή ασπρόμαυρες

Αξίζει να σημειωθεί ότι στα παραπάνω μπορείτε να προσθέσετε περισσότερα ερωτήματα ή να αλλάξετε την κλιμακωσιμότητα των queries (πέρα από το ίδιο, αρκετά ίδιο, σχετικά ίδιο) που θα κατασκευάσετε(π.χ. μια κλίμακα από 1 στα 5 όπου αυτό είναι δυνατόν), μιας και αυτά στην συνέχεια θα χρησιμοποιηθούν για την λειτουργία του recommender. Έτσι μπορείτε να προσθέσετε queries με τα όποια μπορεί να παράγονται καλύτερες συστάσεις  (το αρχείο movie_metadata.csv περιέχει πολλές πληροφορίες ακόμα για κάθε ταινία όπως έτος κυκλοφορίας, βαθμολογία στο imdb, facebοok_likes κ.α.).

Περισσότερες πληροφορίες για το dataset μπορείτε να διαβάσετε σε αυτό το [Link](https://www.kaggle.com/georgefila/movies-metadata):

---

Αξίζει να σημειωθέι πως προσθέσαμε και μερικά δικά μας ερωτήματα.Το πρώτο εκ των 3, αφορά την εύρεση ταινιών που έχουν συγκεντρώσει παρόμοια βαθμολογία από τους χρήστες.Έχει 4 βαθμολογικές κλίμακες ανάλογα τη βαθμολογία (χαμηλή :0-3, μέτρια:3-6, καλή:6-8, πολύ καλή: 8-10).
Το 2ο ερώτημα αφόρα την εύρεση ταινιών με παρόμοιο χρόνο διάρκειας (0-1 ώρα: χαμηλή, 1-2 ώρες: μέτρια, 2+ ώρες: μακρά διάρκεια). 
Το τελευταίο ερώτημα που διατυπώσαμε αφορά την εύρεση ταινιών για τις οποίες παρόμοιος αριθμος χρηστών έχει καταθέσει βαθμολογία.Έχει 3 βαθμολογικές κλίμακες (0-4.000, 4.000-8.000, 8.000+ )

In [ ]:
#---------------------------------------------------------------------------#
#                      1) Common Subject                                    #
#---------------------------------------------------------------------------# 
prolog.assertz('(similar_genre_4(X, Y) :-  genre(X,A), genre(Y,A), genre(X,B), genre(Y,B), genre(X,C), genre(Y,C), genre(X,D), genre(Y,D), A\=B, A\=C, A\=D, B\=C, B\=D, C\=D, X\=Y)')

#---------------------------------------------------------------------------#
#                      2) Less common Subject                               #
#---------------------------------------------------------------------------# 
prolog.assertz('(similar_genre_3(X, Y) :-  genre(X,A), genre(X,B), genre(X,C), genre(Y,A), genre(Y,B), genre(Y,C), X\=Y, A\= B, A\= C, B\=C, not(similar_genre_4(X, Y)) )')

#---------------------------------------------------------------------------#
#                      3) Even Less Common Subject                          #
#---------------------------------------------------------------------------# 
prolog.assertz('(similar_genre_2(X, Y) :-  genre(X,A), genre(X,B), genre(Y,A), genre(Y,B), X\=Y, A\= B, not(similar_genre_3(X, Y)) )')
prolog.assertz('(similar_genre_1(X, Y) :-  genre(X,A), genre(Y,A), X\=Y, not(similar_genre_2(X, Y)) )')

#---------------------------------------------------------------------------#
#                      4) Common Director                                     #
#---------------------------------------------------------------------------# 
prolog.assertz('(common_director(X, Y) :-  director(X,A), director(Y,A), X\=Y )')

#---------------------------------------------------------------------------#
#                      5) Same Plot                                         #
#---------------------------------------------------------------------------# 
prolog.assertz('(similar_keyword_4(X, Y) :-  keyword(X,A), keyword(Y,A), keyword(X,B), keyword(Y,B), keyword(X,C), keyword(Y,C), keyword(X,D), keyword(Y,D), A\=B, A\=C, A\=D, B\=C, B\=D, C\=D, X\=Y)')
prolog.assertz('(similar_keyword_3(X, Y) :-  keyword(X,A), keyword(X,B), keyword(X,C), keyword(Y,A), keyword(Y,B), keyword(Y,C), X\=Y, A\= B, A\= C, B\=C, not(similar_keyword_4(X, Y)) )')

#---------------------------------------------------------------------------#
#                      6) Common Plot                                       #
#---------------------------------------------------------------------------# 
prolog.assertz('(similar_keyword_2(X, Y) :-  keyword(X,A), keyword(X,B), keyword(Y,A), keyword(Y,B), X\=Y , A \= B, not(similar_keyword_3(X, Y)))')
prolog.assertz('(similar_keyword_1(X, Y) :-  keyword(X,A), keyword(X,B), X\=Y , A \= B, not(similar_keyword_2(X, Y)))')

#---------------------------------------------------------------------------#
#                      7) 3 Actors Common                                   #
#---------------------------------------------------------------------------# 
prolog.assertz('(common_actors_3(X, Y) :-  actors(X,A), actors(X,B), actors(X,C), actors(Y,A), actors(Y,B), actors(Y,C), X\=Y, A \= B, A \= C, B \= C)')

#---------------------------------------------------------------------------#
#                      8) 2 Actors Common                                   #
#---------------------------------------------------------------------------#
prolog.assertz('(common_actors_2(X, Y) :-  actors(X,A), actors(X,B), actors(Y,A), actors(Y,B), X\=Y, A \= B, not(common_actors_3(X, Y)) )')

#---------------------------------------------------------------------------#
#                      9) 1 Actors Common                                   #
#---------------------------------------------------------------------------#
prolog.assertz('(common_actors_1(X, Y) :-  actors(X,A), actors(Y,A), X\=Y, not(common_actors_2(X, Y)) )')

#---------------------------------------------------------------------------#
#                      10) Common Language                                  #
#---------------------------------------------------------------------------#
prolog.assertz('(common_language(X, Y) :- language(X,D), language(Y,D), X \= Y)')

#---------------------------------------------------------------------------#
#                      11) Same color                                       #
#---------------------------------------------------------------------------#
prolog.assertz('(same_color(X,Y) :- color(X, C), color(Y, C), X \= Y )')



#---------------------------------------------------------------------------#
#                      12) Our Querries                                     #
#---------------------------------------------------------------------------#
# Same Rating (4 categories depending on the score)
prolog.assertz('''(same_rating(X, Y) :- vote(X,C),   vote(Y,C),   X \= Y )''')

# Same duration (3 categories, over 2 hours,1-2 hours and under 1 hour)
prolog.assertz('''(same_duration(X, Y) :- duration(X,C),   duration(Y,C),   X \= Y )''')

# Same volume of numbers voted (3 categories depending on the number of votes)
prolog.assertz('''(number_of_users_voted(X, Y) :- users_voted(X,C),   users_voted(Y,C),   X \= Y )''')
 

Στη συνέχεια θα προχωρήσουμε σε έλεγχο της λειτουργίας των querries.

In [ ]:
#---------------------------------------------------------------------------#
#                       Testing                                             #
#---------------------------------------------------------------------------#

xs = ["similar_genre_4('","similar_genre_3('","similar_genre_2('","similar_genre_1('",
      "similar_keyword_3('","similar_keyword_4('","common_actors_1('","common_actors_2('",
      "common_actors_3('","common_director('","common_language('", "same_color('", "same_duration('","same_rating('",
      "number_of_users_voted('"]
def TEST(movie):
  for x in xs: 
    querry = prolog.query(x + movie +"',M)")
    result = set()
    for solution in querry:
        result.add(solution["M"])
    print(x.split("(")[0],list(set(result))[:5])
  return


In [ ]:
print('Unbreakable\n---------------------------------------------------------------\n')
TEST("Unbreakable")
print('\n\nThe Dark Knight Rises\n---------------------------------------------------------------\n')
TEST("The Dark Knight Rises")

Unbreakable
---------------------------------------------------------------

similar_genre_4 []
similar_genre_3 ['Chronicle', 'Southland Tales', 'I Am Legend', '10 Cloverfield Lane', 'Victor Frankenstein']
similar_genre_2 ['The I Inside', 'Planet of the Apes', 'Star Trek V: The Final Frontier', 'Crimson Tide', 'After.Life']
similar_genre_1 ['Red Eye', 'AWOL-72', 'Maurice', 'Next Stop Wonderland', 'Serial Mom']
similar_keyword_3 ['Superman Returns']
similar_keyword_4 []
common_actors_1 ['Adore', 'Jumper', 'What Just Happened', 'Eves Bayou', 'Kiss of Death']
common_actors_2 []
common_actors_3 []
common_director ['The Visit', 'After Earth', 'The Sixth Sense', 'The Happening', 'The Last Airbender']
common_language ['Red Eye', 'AWOL-72', 'Shrek the Third', 'Maurice', 'Intolerance']
same_color ['Shrek the Third', 'Paddington', 'Planet of the Apes', 'Crimson Tide', 'Killer Elite']
same_duration ['Red Eye', 'AWOL-72', 'Shrek the Third', 'Paddington', 'Next Stop Wonderland']
same_rating ['Red E

---
# **2ο Μέρος: Recommendation System με βάση μόνο τα χαρακτηριστικά των ταινιών.**
---

## Εκφώνηση


Στο σημείο αυτό καλείστε με βάση αυτά που κάνατε στο μέρος 1 να κατασκευάσετε queries τα όποια θα σας επιστρέφουν παρόμοιες (σε χαρακτηριστικά) ταινίες. Τα ερωτήματα αυτά θα είναι κλιμακούμενα, δηλαδή θα υπάρχουν ερωτήματα που επιστρέφουν αρκετά κοινές ταινίες αλλά και που επιστρέφουν λιγότερο και λιγότερο κοινές (σε μια κλίμακα π.χ. από 1 σε 5). Για παράδειγμα:

```
find_simmilar_movies_5("Pirates Of The Caribbean", M).
M = "Pirates Of The Caribbean: On Stranger Tides"
M = "The Chronicles Of Narnia"
M = "Prince Of Persia: The Sands Of Time"
...
```
Για παράδειγμα, το παραπάνω ερώτημα θα επιστρέφει αρκετά κοινές σε περιεχόμενο ταινίες με την ταινια "Pirates Of The Caribbean". Θα υπάρχουν και αντίστοιχα ερωτήματα που θα βρίσκουν λιγότερο όμοιες ταινίες. Ο δείκτης ομοιότητας των ταινιών είναι αυθαίρετος και μπορείτε να τον ορίσετε εσείς όπως θέλετε, αρκεί να υπάρχει κάποια λογική σύνδεση με τα δεδομένα που περιέχονται στο αρχείο movies_metadata.csv. 

Συνεπώς η συνάρτηση που θα κάνει τις προτάσεις (recommendation) με είσοδο μια ταινία πρέπει να επιστρέφει-εκτυπώνει μια λίστα με τις προτεινόμενες ταινίες κατά φθίνουσα σειρά ομοιότητας. 

##Simple Recommender
---



Mε βάση τους κανόνες που κατασκευάστηκαν στο Μέρος 1 θα κατασκευαστούν κατηγορήματα για την ομοιότητα ταινιών. 


Στην παρακάτω θεώρηση, ταινίες ομοιότητας τύπου 5 θεωρούνται αρκετά όμοιες,ενώ καθώς μειώνεται ο αριθμός η ομοιότητα αυξάνεται (ομοιότητα 5 > ομοιότητα 4)

In [ ]:
prolog.assertz('(find_sim_5(X, Y) :- similar_genre_4(X, Y),common_language(X, Y),similar_keyword_4(X, Y); common_actors_1(X,Y),similar_genre_3(X, Y),similar_keyword_4(X, Y), common_director(X, Y),common_language(X, Y);common_actors_3(X, Y),common_language(X, Y),similar_keyword_2(X, Y),similar_genre_3(X, Y))')
prolog.assertz('(find_sim_4(X, Y) :- common_actors_1(X, Y), similar_genre_3(X, Y),common_language(X, Y),similar_keyword_3(X, Y);similar_keyword_4(X, Y),common_language(X, Y),similar_genre_2(X, Y);common_actors_3(X, Y),common_language(X, Y),similar_keyword_1(X, Y),similar_genre_1(X, Y))')
prolog.assertz('(find_sim_3(X, Y) :- similar_genre_3(X, Y),common_language(X, Y),similar_keyword_2(X, Y); similar_genre_2(X, Y),common_language(X, Y),similar_keyword_3(X, Y))')
prolog.assertz('(find_sim_2(X, Y) :- similar_keyword_1(X, Y),common_language(X, Y),similar_genre_2(X, Y);similar_keyword_2(X, Y),common_language(X, Y),similar_genre_1(X, Y) )')
prolog.assertz('(find_sim_1(X, Y) :- similar_genre_1(X, Y),similar_keyword_1(X, Y),common_language(X, Y) )')

Παρακάτω ακολουθεί ενα απλό recommender system το οποίο βασίζεται στα δεδομένα ταινιών που μας δώθηκαν.Προκειμένου να παρουσιάσουμε όσο το δυνατόν πιο σχετικές ταινίες, αρχικά αναζητούνται ταινίες ομοιότητας 1.Εάν δεν βρεθεί καμία,τότε προχωρούμε στην αναζήτηση ταινιών ομοιότητας 2 κ.ο.κ.

In [ ]:
def simple_recommender(movie):
  def check(querry):
    s = set()
    for solution in querry:
      m = solution['M'] 
      if m not in s:
        s.add(solution['M'])
    return s

  # Similarity will be used in part-3
  similarity = 0
  # A list containing all recommended movies
  answers = []
  s = set()
  q = prolog.query("find_sim_5('" + movie +"',M)")
  s = check(q)
  if (s): 
    similarity = 5
    answers = s
    q.close()
    return answers,similarity
  else:
    q = prolog.query("find_sim_4('" + movie +"',M)")
    s = check(q)
    if (s): 
      similarity = 4
      answers = s
      q.close()
      return answers,similarity 
    else:
      q = prolog.query("find_sim_3('" + movie +"',M)")
      s = check(q)
      if (s): 
        similarity = 3
        answers = s
        q.close()
        return answers,similarity 
      else:
        q = prolog.query("find_sim_2('" + movie +"',M)")
        s = check(q)
        if (s): 
          similarity = 2
          answers = s
          q.close()
          return answers,similarity 
        else:
          q = prolog.query("find_sim_1('" + movie +"',M)")
          s = check(q)
          if (s): 
            similarity = 1
            answers = s
            q.close()
            return answers,similarity 
          else:
            return answers,similarity

## Functionality Test
---

Δομιμάζουμε το recommender system σε μερικές ταινίες.Ρυθμίζουμε το σύστημα στην παρούσα στιγμή προκειμένου να μας προτείνει μέχρι 3 ταινίες. 
Αυτή τη στιγμή δε μας ενδιαφέρει να μας προτείνει το σύστημα πολλές ταινίες, αλλά απλώς να ελέγξουμε τη λειτουργία του.Εάν είναι επιθυμητή η πρόταση πολλών ταινιών θα μπορούσε να ρυθμιστεί κατάλληλα.

In [ ]:
max_rec_movies = 3

sample_movies = ["Batman", "Spectre", "Avatar","Network",
                 "Jaws 2","Titanic","King Kong","Toy Story 2",
                 "Harry Potter and the Prisoner of Azkaban"]

for x in sample_movies:  
  rec_mov,similarity = simple_recommender(x)
  rec_mov = list(rec_mov)[:max_rec_movies]
  print("You like "+ x +", we also recommend",end=": ")
  print(', '.join(list(rec_mov)))


You like Batman, we also recommend: Superman
You like Spectre, we also recommend: Quantum of Solace, Johnny English Reborn, Skyfall
You like Avatar, we also recommend: Predator, Green Lantern, Enders Game
You like Network, we also recommend: The Lives of Others, 11:14, The Doors
You like Jaws 2, we also recommend: Jaws
You like Titanic, we also recommend: Pompeii
You like King Kong, we also recommend: Pirates of the Caribbean: Dead Mans Chest
You like Toy Story 2, we also recommend: Night at the Museum, Megamind, The Brave Little Toaster
You like Harry Potter and the Prisoner of Azkaban, we also recommend: Harry Potter and the Goblet of Fire, Aladdin, Harry Potter and the Order of the Phoenix


In [ ]:
max_rec_movies = 3

sample_movies = ["Batman", "Spectre", "Avatar","Network",
                 "Jaws 2","Titanic","King Kong","Toy Story 2",
                 "Harry Potter and the Prisoner of Azkaban"]

for x in sample_movies:  
  rec_mov,similarity = simple_recommender(x)
  rec_mov = list(rec_mov)[:max_rec_movies]
  print("You like "+ x +", we also recommend",end=": ")
  print(', '.join(list(rec_mov)))


Για ένα τόσο απλό σύστημα τα αποτελέσματα κρίνονται ως ικανοποιητικά, καθώς παρατηρούμε μια σχετική συνάφεια στις ταινίες που προτείνονται. Για παράδειγμα στην ταινια "Βatman" η πρόταση "Superman" είναι πολύ λογική και στην ταινία "Τoy story 2" προτείνονται επίσης ταινίες κινουμένων σχεδίων για παιδιά.Η γενική θεματολογία των προτεινόμενων ταινιών είναι αρκετά κοντά με την αρχική.

---
# **3ο Μέρος: Recommendation System Με βάση τις προτιμήσεις-Αξιολογήσεις του χρήστη-Εκπαίδευση και Πρόβλεψη**
---




## Εκφώνηση

Σε αυτό το σημείο θα εργαστείτε με τα αρχεία ratings τα όποια περιέχουν αξιολογήσεις (από 1 μέχρι 5) για τις παραπάνω ταινίες. Το προηγούμενο σύστημα συστάσεων προτείνει στον χρήστη ταινίες αποκλειστικά με βάση την ομοιότητά τους. Σε αυτό το σημείο θα γίνει μια αναβάθμιση του συστήματος έτσι ώστε να παράγονται καλύτερες  συστάσεις οι όποιες θα λαμβάνουν υπόψιν και τις προτιμήσεις του χρήστη, οι όποιες θα εξάγονται από τις αξιολογήσεις που έχει κάνει μέχρι στιγμής. 

Η εκπαίδευση του recommender θα γίνεται ως εξής:

Για κάθε ταινία θα υπάρχει ένα score το όποιο αρχικά θε είναι ίσο με 0 και θα διαμορφώνεται από τις αξιολογήσεις κάθε user. Έτσι για έναν χρήστη με βάση τις αξιολογήσεις που υπάρχουν στο αρχείο train_ratings θα πρέπει:

1.	Για κάθε ταινία που έχει βαθμολογήσει να βρίσκονται οι κοινές ταινίες ανά κλίμακα και στο μέχρι τώρα σκορ κάθε παρόμοιας ταινίας θα προστίθεται ένα βάρος το όποιο θα μπορούσε να είναι το ποσοστό ομοιότητας της ταινίας (δηλαδή ένα βάρος για κάθε κλίμακα) επί τον βαθμό που έχει βάλει ο χρήστης για την αρχική ταινία. Στη συνέχεια, ανάλογα με το σκορ που έχει σχηματιστεί για κάθε ταινία, θα επιλέγεται αν αυτή θα μπορούσε να είναι προτεινόμενη για τον χρήστη.

Η λογική πίσω από την παραπάνω διαδικασία είναι ότι παρόμοιες ταινίες θα έχουν ανάλογο βαθμό. Για παράδειγμα αν ένας χρήστης έχει αξιολογήσει αρκετές  ταινίες οι όποιες είναι sci-fiction με 5/5 τότε μια ταίνια sci-fiction την όποια δεν έχει δεί λογικά θα του αρέσει και θα έπρεπε να την προτείνουμε.

Μετά την εκπαίδευση του συστήματος σας καλείστε να δοκιμάσετε τον recommender που κατασκευάσατε στην πράξη. Για τον σκοπό αυτό θα φορτώσετε το αρχείο train_ratings.csv όπου περιέχονται οι αξιολογήσεις του ίδιου χρήστη για άλλες ταινίες. Το σύστημά σας πρέπει να προβλέπει αν μια ταινία θα πρέπει να προταθεί στον χρήστη. Μια ταίνια θα έπρεπε να έχει προταθεί στον  χρήστη, αν έχει βαθμό μεγαλύτερο του 3. Συνεπώς για την επίβλεψη του συστήματός σας θα πρέπει για κάθε μια από τις ταινίες του αρχείου test_ratings.csv να επιστρέφεται 1 ή 0 αν η ταινία θα έπαιρνε βαθμό μεγαλύτερο του 3 ή όχι, δηλαδή αν θα έπρεπε να την είχαμε προτείνει ή όχι.

Στην συνέχεια, σε συνδιασμό με τις πραγματικές απαντήσεις του χρήστη θα αξιολογήσετε το σύστημά σας χρησιμοποιόντας τις μετρικές: precision, recall, f1 οι οποίες είναι οι πλέον γνωστές μετρικές και ευρέως χρησιμοποιούμενες τεχνικές για την επίβλεψη-μέτρηση απόδοσης ανάλογων συστημάτων.

1. Precision: Δείχνει πόσο ακριβές είναι το σύστημα. Υπολογίζει πόσα από τα στιγμιότυπα τα όποια προβλέψαμε ότι ανήκουν σε μια κλάση όντως ανήκουν σε αυτή. Η μετρική αυτή μας δίνει μια εικόνα σχετικά με τον αριθμό των ταινίων που προβλέψαμε ως προτεινόμενες ενώ δεν θα έπρεπε.

2. Recall: Υπολογίζει πόσα από τα στιγμιότυπα που ανήκουν σε μια κλάση (π.χ. προτεινόμενες ταινίες) προβλέφθηκαν σωστά.

3. F1: Είναι ένας μέσος μεταξύ των παραπάνω δυο μετρικών, έτσι ώστε να διατηρείται μια ισορροπία μεταξύ τους. Υπολογίζεται από την παρακάτω σχέση:

$$F_1=2\frac{Precision\times{Recall}}{Precision+Recall}$$

Οι παραπάνω συναρτήσεις παρέχονται από την βιβλιοθήκη scikit-learn.

Τέλος για την καλύτερη επίβλεψη του συστήματος σας μπορείτε να εκπαιδεύσετε  τον recommender σας με ένα υποσύνολο ταινίων (π.χ. 10, 20, 50, ...) για να μελετήσετε κατά πόσο σας βοηθούν οι επιπλέον αξιολογήσεις κάθε φορά (δηλαδή κατά πόσο βελτιώνονται οι παραπάνω μετρικές στο test set). Έτσι π.χ. μπορείτε να εντοπίσετε περιπτώσεις όπως για παράδειγμα ότι με έναν recommender μπορεί να μην επιτυγχάνετε πολύ υψηλό σκορ όσο με άλλους, αλλά το βέλτιστο σκόρ σας επιτυγχάνεται πολύ γρηγόρα π.χ. με μόνο 10 ταινίες αντί 100.

## Προβλήματα με το PySwip

Σε αυτό το σημείο αντιμετωπίσαμε έντονα προβλήματα στη χρήση του PySwip.Συγκεκριμένα, σε querries που επιστρέφουν strings (όπως είναι τα δικά μας) ο compiler κατά τη διάρκεια της αναζήτησης παράγει προσωρινά strings τα οποία δεν απελευθερώνει στη συνέχεια της εκτέλεσης του προγράμματος.Ως αποτέλεσμα,το runtime crashάρει.Λίγο πριν τον τερματισμό, λαμβάνουμε το warning "Too many stacked strings".Πιθανές λύσεις που δοκιμάστηκαν:


*   Τρέξιμο του κώδικα τοπικά στο Jupyter, σε .py, σε Anaconda enviroment και σε cloud στην υπηρεσία Kaggle
*   [Aύξηση του stack size της prolog.](https://www.swi-prolog.org/FAQ/StackSizes.html)
*   Xρήση κάποιον [μακροεντολών](https://www.swi-prolog.org/pldoc/man?CAPI=PL_STRINGS_MARK) για την αποφυγή του προβλήματος
*   Αλλαγή των querries ώστε να επιστρέφουν λιγότερες ταινίες, άρα και να γεμίζεται λιγότερο η μνήμη

Από τα παραπάνω, το τρέξιμο του κώδικα τοπικά δεν απέδωσε καθώς το ίδιο πρόβλημα εξακολούθησε να υφίσταται.Mε την αύξηση του stack_size με την εντολή 
":-set_prolog_flag(stack_limit, 5000000000)." το πρόβλημα δε φάνηκε να βελτιώνεται,πιθανώς δεν κάνουμε σωστό χειρισμό της εντολής.
  H χρήση του block μακροεντολών δεν απέδωσε, καθώς όποτε χρησιμοποιήθηκε, ο compiler μετά δεν αναγνώριζε τα querries που προσπαθούσαμε να φορτώσουμε.

  Για να παρακάμψουμε το πρόβλημα αρχικά τρέχαμε τις αναζητήσεις για ένα υποσύνολο των ταινιών αποτελούμενο μόνο από 10 ταινίες.Καθώς η συνάρτηση "train recommender" μπορεί να δεχτεί και ως όρισμα το movies_score για ένα υποσύνολο δεδομένων, τρέχαμε την εκπαίδευση για 10 μόνο ταινίες,αποθηκεύαμε τα αποτελέσματα στο google drive μέσω του εργαλείου joblib, και ύστερα κάναμε restrat runtime.Έχουμε προσαρμόσει κατάλληλα το επόμενο κελί ώστε να φορτώνει το movies_score από το google drive και να συνεχίζει την εκπαίδευση από εκεί όπου είχε μέινει.

  Η τελική λύση προήλθε με την χρήση του "distict" .Από την στιγμή που προστέθηκε στα querries μας σταμάτησε η εμφάνιση του προβλήματος.



## Σύστημα συστάσεων

Αρχικά μελετάμε τις αξιολογήσεις κάθε χρήστη για να καταλάβουμε την δομή και τις πληροφορίες κάθε αρχείου.

In [ ]:
train_ratings.head()

,Unnamed: 0,userId,movieId,movie_title,rating,timestamp
0,78477,547,235,Stand by Me,4.0,974780707
1,78747,547,1251,Letters from Iwo Jima,5.0,974809897
2,78808,547,1391,Y Tu Mamá También,0.5,1076809696
3,78584,547,824,Moulin Rouge!,2.0,1076968814
4,80322,547,51540,Horrible Bosses,4.0,1192372083


In [ ]:
test_ratings.head()

,Unnamed: 0,userId,movieId,movie_title,rating,timestamp
0,78436,547,11,Star Wars,3.0,1418149949
1,78437,547,14,American Beauty,3.5,1053171195
2,78438,547,16,Dancer in the Dark,4.0,986694981
3,78441,547,25,Jarhead,4.5,1053087861
4,78452,547,111,Scarface,5.0,974809870


### Επανακαθορισμός της Simple Recommender

Εξ'αιτίας όσων περιγράφονται παραπάνω κρίθηκε αναγκαίος ο εκ νέου καθορισμός των querries αναζήτησης όμοιων ταινιών,καθώς και ο καθορισμός της συνάρτησης "simple_recommender".

In [ ]:
def find_movies(movie, similarity):
    q = prolog.query("find_sim_"+similarity+"('" + movie +"',M)")
    ans = []
    for soln in q:
        ans.append((soln["M"],similarity))
    ans = set(ans)
    return ans

def simple_recommender(movie):
  ans = set()
  temp_ans = set()
  temp_ans = find_movies(movie, "5")
  ans.update((temp_ans))
  temp_ans = find_movies(movie, "4")
  ans.update((temp_ans))
  temp_ans = find_movies(movie, "3")
  ans.update((temp_ans))
  temp_ans = find_movies(movie, "2")
  ans.update((temp_ans))
  temp_ans = find_movies(movie, "1")
  ans.update((temp_ans))
  return list(ans)

In [ ]:
print(simple_recommender("Batman"))

[('Dead Man on Campus', '3'), ('Boyz n the Hood', '3'), ('Six Days Seven Nights', '3'), ('Just Go with It', '3'), ('Caddyshack', '3'), ('Event Horizon', '3'), ('Star Trek: The Motion Picture', '2'), ('Risen', '3'), ('The Bucket List', '3'), ('Return to the Blue Lagoon', '3'), ('Faster', '2'), ('Me You and Five Bucks', '3'), ('X2', '3'), ('Alvin and the Chipmunks: The Road Chip', '3'), ('The Other End of the Line', '3'), ('The Life Aquatic with Steve Zissou', '2'), ('The Dead Girl', '3'), ('The Way Way Back', '3'), ('Falcon Rising', '2'), ('Welcome to Collinwood', '3'), ('Dawn of the Planet of the Apes', '2'), ('Street Fighter', '2'), ('Dinner Rush', '3'), ('My Bosss Daughter', '3'), ('Speedway Junky', '3'), ('Fat Albert', '2'), ('Big Trouble in Little China', '2'), ('Killer Elite', '2'), ('The Jackal', '3'), ('The Prince of Egypt', '2'), ('The Mist', '3'), ('Rambo: First Blood Part II', '2'), ('Madagascar 3: Europes Most Wanted', '3'), ('Beverly Hills Chihuahua', '3'), ('R.I.P.D.', '2'

Eπιπλέον, είναι απαραίτητος ο εκ νέου καθορισμός των querries αναζήτησης διότι τα προηγούμενα δεν φέρουν πλέον ικανοποιητικά αποτελέσματα.Κάνουμε consult ένα καινούργιο αρχείο αναζητήσεων.

In [ ]:
num = 4
prolog.consult(path + "db"+str(num)+".pl")

# Επίσης μπορούμε να κάνουμε consult ένα έτοιμο αρχείο στον κόσμο όπως παρακάτω
#  prolog.consult(path +"db.pl")
train_ratings = pd.read_csv(path + "train_ratings.csv")
test_ratings = pd.read_csv(path + "test_ratings.csv")

Τα περιεχόμενα του αρχείου db4.pl παρουσιάζονται παρακάτω: 

```
# db4.pl
prolog.assertz('find_sim_5(X, Y) :- distinct((similar_keyword_4(X, Y); common_actors_2(X, Y), similar_genre_3(X, Y),common_director(X, Y) )))')
prolog.assertz('find_sim_4(X, Y) :- distinct(((common_actors_2(X, Y);common_director(X, Y)),similar_keyword_4(X, Y)), not(find_sim_5(X, Y)) ))')
prolog.assertz('find_sim_3(X, Y) :- distinct((similar_genre_1(X, Y); common_actors_2(X, Y); similar_keyword_3(X, Y), not(find_sim_4(X, Y)) )) )')
prolog.assertz('find_sim_2(X, Y) :- distinct((similar_keyword_3(X, Y);similar_genre_2(X, Y)); not(find_sim_3(X, Y)) ))')
prolog.assertz('find_sim_1(X, Y) :- distinct((similar_genre_1(X, Y); similar_keyword_1(X, Y) ), not(find_sim_2(X, Y)) ))')

```

 

Όπως αναφέρθηκε και παραπάνω, το keyword "distinct" ήταν ο καθοριστικός παράγοντας για την εξάλειψη του προβλήματος με τη μνήμη που αντιμετωπίσαμε.

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

similarity_weights = { 5:10, 4:7, 3:3, 2:1, 1:0}
rating_weights = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10, 0:-20} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

def train_recommender(ratings, rating_weights, similarity_weights, start=0, end = -1, movie_score = {}):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train,
    ενώ παράλληλα μπορούμε να αρχικοποιήσουμε και το movie_score το οποίο αποθηκεύονται τα score κάθε ταινίας.
    Έτσι αν έχουμε κάνει train τον recommender μας για τις πρώτες 10 ταινίες, μπορούμε να συνεχίσουμε για τις 
    υπόλοιπες χωρίς κάθε φορά να ξανά-διαμορφώνουμε το score για αυτές. Αυτό θα σας βοηθήσει στην εκτέλεση και χρονικά όταν ελέγχετε την 
    απόδοση σε υποσύνολα των ratings π.χ. πρώτες 10 ταινίες έπειτα τις επόμενες 20 κ.ο.κ.
    """
    if end == -1:
        end = len(ratings)

    ratings = ratings.iloc[range(start, end)]
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')
        print("rating:",rating,", movie:",movie)

        # similar_movies=[]
        similar_movies=simple_recommender(movie)
        # print(similar_movies)
        for s in similar_movies:
          # print("s=",s)
          if s[0] not in movie_score:
              movie_score[s[0]] = rating_weights[int(rating)] * similarity_weights[int(s[1])]
          else:
              # print("else")
              movie_score[s[0]] += rating_weights[int(rating)] * similarity_weights[int(s[1])] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 1
def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > 0)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
print(len(train_ratings))

100


## Έλεγχος λειτουργικότητας

### Εκπαίδευση με υποσύνολα των δεδομένων

Αρχικά εκπαιδεύσαμε το μοντέλο βήμα-βήμα, διότι δεν είχαμε λύσει ακόμα το πρόβλημα με όνομα "too many stacked stings".Θα αποδείξουμε πως η μέθοδος της εκπαίδευσης με υποσύνολα των συνολικών ταινιών,καθώς και η μέθοδος της εκπαίδευσης κατευθείαν του συνόλου των ταινιών οδηγούν στα ίδια αποτελέσματα.

In [ ]:
import joblib
filename1 = 'movie_score.pkl'

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, 0, 10, movie_score)
print(movie_score)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + filename1, compress = True)

rating: 4.0 , movie: Stand by Me
rating: 5.0 , movie: Letters from Iwo Jima
rating: 0.5 , movie: Y Tu Mamá También
rating: 2.0 , movie: Moulin Rouge!
rating: 4.0 , movie: Horrible Bosses
rating: 5.0 , movie: Dances with Wolves
rating: 3.5 , movie: Along Came a Spider
rating: 2.0 , movie: The Sixth Sense
rating: 5.0 , movie: The Forbidden Kingdom
rating: 1.0 , movie: Notting Hill

{'Courageous': -45, 'Mozarts Sister': -45, 'The Frozen Ground': 15, 'Naturally Native': -45, 'Blue Car': -45, 'The Pink Panther': 30, 'Flyboys': 0, 'The Holy Girl': -45, 'Journey from the Fall': -45, 'Bodyguards and Assassins': -45, 'Victor Frankenstein': -30, 'Knocked Up': 45, 'Faith Like Potatoes': -45, 'Rust': -45, 'To Save A Life': -45, '#Horror': -45, 'Eddie the Eagle': 0, 'The Best Years of Our Lives': 60, 'Bloody Sunday': -75, 'La Bamba': -30, 'The Quiet': -30, 'Without Limits': -45, 'Martha Marcy May Marlene': -30, 'Georgia Rule': 45, 'Valley of the Wolves: Iraq': -75, 'The Hundred-Foot Journey': -45, 

['/gdrive/My Drive/tn2/movie_score.pkl']

In [ ]:
my_object = joblib.load(path + 'movie_score.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, 10, 20, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_10_20.pkl', compress = True)

rating: 2.5 , movie: Transformers: Revenge of the Fallen
rating: 3.0 , movie: Mr. Beans Holiday
rating: 4.5 , movie: Match Point
rating: 4.5 , movie: The Island
rating: 4.5 , movie: Team America: World Police
rating: 2.0 , movie: Dragonslayer
rating: 3.5 , movie: Somethings Gotta Give
rating: 1.0 , movie: Rocky Balboa
rating: 3.5 , movie: X-Men Origins: Wolverine
rating: 4.0 , movie: Cinderella Man

{'precision': 0.44776119402985076, 'recall': 0.4166666666666667, 'f1': 0.43165467625899284}


['/gdrive/My Drive/tn2/movie_score_10_20.pkl']

In [ ]:
my_object = joblib.load(path + 'movie_score_10_20.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, 20, 30, movie_score = my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_20_30.pkl', compress = True)

rating: 3.0 , movie: Army of Darkness
rating: 2.0 , movie: Notes on a Scandal
rating: 2.0 , movie: My Super Ex-Girlfriend
rating: 4.0 , movie: Superman
rating: 3.0 , movie: Jekyll and Hyde ... Together Again
rating: 3.0 , movie: Psycho
rating: 4.5 , movie: Confessions of a Dangerous Mind
rating: 3.0 , movie: Lawrence of Arabia
rating: 3.0 , movie: The Curse of the Jade Scorpion
rating: 2.0 , movie: Payback

{'precision': 0.4074074074074074, 'recall': 0.3055555555555556, 'f1': 0.3492063492063492}


['/gdrive/My Drive/tn2/movie_score_20_30.pkl']

In [ ]:
my_object = joblib.load(path + 'movie_score_20_30.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, 30, 40)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_30_40.pkl', compress = True)

rating: 5.0 , movie: Hannibal Rising
rating: 5.0 , movie: Amélie
rating: 4.0 , movie: Deconstructing Harry
rating: 4.0 , movie: The French Connection
rating: 3.5 , movie: Alvin and the Chipmunks
rating: 4.0 , movie: Harry Potter and the Philosophers Stone
rating: 3.0 , movie: Spellbound
rating: 3.0 , movie: Friends with Money
rating: 1.0 , movie: 54
rating: 5.0 , movie: Terminator 3: Rise of the Machines

{'precision': 0.5327868852459017, 'recall': 0.9027777777777778, 'f1': 0.6701030927835052}


['/gdrive/My Drive/tn2/movie_score_30_40.pkl']

In [ ]:
val = 40
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+10, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+10)+'.pkl', compress = True)

rating: 2.0 , movie: About Schmidt
rating: 5.0 , movie: Solaris
rating: 5.0 , movie: Madagascar
rating: 3.5 , movie: Open Season
rating: 4.0 , movie: Flash Gordon
rating: 4.0 , movie: Interview with the Vampire
rating: 4.0 , movie: Dear Frankie
rating: 2.5 , movie: Freddy vs. Jason
rating: 3.0 , movie: Bill & Teds Bogus Journey
rating: 5.0 , movie: Escape from New York

{'precision': 0.5112781954887218, 'recall': 0.9444444444444444, 'f1': 0.6634146341463415}


['/gdrive/My Drive/tn2/movie_score_40_50.pkl']

In [ ]:
val = 50
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+10, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+10)+'.pkl', compress = True)

rating: 3.0 , movie: Lions for Lambs
rating: 3.0 , movie: Seven Samurai
rating: 4.0 , movie: Shopgirl
rating: 2.5 , movie: Maurice
rating: 4.0 , movie: The Legend of Suriyothai
rating: 2.0 , movie: Being John Malkovich
rating: 5.0 , movie: A Woman, a Gun and a Noodle Shop
rating: 4.5 , movie: The Karate Kid
rating: 1.5 , movie: Mean Streets
rating: 3.5 , movie: Titanic

{'precision': 0.5112781954887218, 'recall': 0.9444444444444444, 'f1': 0.6634146341463415}


['/gdrive/My Drive/tn2/movie_score_50_60.pkl']

In [ ]:
val = 60
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+9, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+9)+'.pkl', compress = True)

rating: 3.0 , movie: Manderlay
rating: 3.0 , movie: Raiders of the Lost Ark
rating: 4.0 , movie: Highlander: The Final Dimension
rating: 4.0 , movie: This Is 40
rating: 2.0 , movie: Wasabi
rating: 4.0 , movie: Pandoras Box
rating: 4.0 , movie: Barry Lyndon
rating: 0.5 , movie: Rebecca
rating: 3.5 , movie: To Kill a Mockingbird

{'precision': 0.5, 'recall': 0.8194444444444444, 'f1': 0.6210526315789474}


['/gdrive/My Drive/tn2/movie_score_60_69.pkl']

In [ ]:
val = 70
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val-1)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+10, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+10)+'.pkl', compress = True)

rating: 5.0 , movie: Space Jam
rating: 3.5 , movie: License to Wed
rating: 2.5 , movie: My Best Friends Wedding
rating: 5.0 , movie: The Talented Mr. Ripley
rating: 4.0 , movie: Shine a Light
rating: 1.0 , movie: Alexander
rating: 1.0 , movie: Under Siege 2: Dark Territory
rating: 4.5 , movie: U-571
rating: 3.0 , movie: Kindergarten Cop
rating: 4.0 , movie: The Transporter

{'precision': 0.5307692307692308, 'recall': 0.9583333333333334, 'f1': 0.6831683168316831}


['/gdrive/My Drive/tn2/movie_score_70_80.pkl']

In [ ]:
val = 80
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+10, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+10)+'.pkl', compress = True)

rating: 0.5 , movie: Live and Let Die
rating: 0.5 , movie: Amadeus
rating: 1.5 , movie: The Butterfly Effect
rating: 2.0 , movie: The Last Emperor
rating: 3.5 , movie: In Time
rating: 2.5 , movie: Rosemarys Baby
rating: 4.5 , movie: Mission: Impossible
rating: 3.0 , movie: Flashdance
rating: 3.0 , movie: Wag the Dog
rating: 5.0 , movie: The First Wives Club

{'precision': 0.527027027027027, 'recall': 0.5416666666666666, 'f1': 0.5342465753424657}


['/gdrive/My Drive/tn2/movie_score_80_90.pkl']

In [ ]:
val = 90
my_object = joblib.load(path + 'movie_score_'+str(val-10)+'_'+str(val)+'.pkl')

movie_score = train_recommender(train_ratings, rating_weights, similarity_weights, val, val+10, movie_score= my_object)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))

joblib.dump(movie_score, path + 'movie_score_'+str(val)+'_'+str(val+10)+'.pkl', compress = True)

rating: 4.5 , movie: Charlie Wilsons War
rating: 2.5 , movie: Along Came Polly
rating: 5.0 , movie: The Wedding Planner
rating: 5.0 , movie: Mission to Mars
rating: 4.0 , movie: The Kite Runner
rating: 3.5 , movie: Aliens vs Predator: Requiem
rating: 2.5 , movie: Romeo + Juliet
rating: 3.5 , movie: Star Trek: Generations
rating: 4.0 , movie: Rambo III
rating: 3.0 , movie: Gothika

{'precision': 0.5113636363636364, 'recall': 0.625, 'f1': 0.5625}


['/gdrive/My Drive/tn2/movie_score_90_100.pkl']

### Εκπαίδευση στο σύνολο των δεδομένων

Εκπαιδεύουμε το μοντέλο με training data όλες τις ταινίες.

In [ ]:
movie_score = train_recommender(train_ratings, rating_weights, similarity_weights)
real, pred = predict_example(test_ratings, movie_score)
print (get_metrics(real, pred))


rating: 4.0 , movie: Stand by Me
rating: 5.0 , movie: Letters from Iwo Jima
rating: 0.5 , movie: Y Tu Mamá También
rating: 2.0 , movie: Moulin Rouge!
rating: 4.0 , movie: Horrible Bosses
rating: 5.0 , movie: Dances with Wolves
rating: 3.5 , movie: Along Came a Spider
rating: 2.0 , movie: The Sixth Sense
rating: 5.0 , movie: The Forbidden Kingdom
rating: 1.0 , movie: Notting Hill
rating: 2.5 , movie: Transformers: Revenge of the Fallen
rating: 3.0 , movie: Mr. Beans Holiday
rating: 4.5 , movie: Match Point
rating: 4.5 , movie: The Island
rating: 4.5 , movie: Team America: World Police
rating: 2.0 , movie: Dragonslayer
rating: 3.5 , movie: Somethings Gotta Give
rating: 1.0 , movie: Rocky Balboa
rating: 3.5 , movie: X-Men Origins: Wolverine
rating: 4.0 , movie: Cinderella Man
rating: 3.0 , movie: Army of Darkness
rating: 2.0 , movie: Notes on a Scandal
rating: 2.0 , movie: My Super Ex-Girlfriend
rating: 4.0 , movie: Superman
rating: 3.0 , movie: Jekyll and Hyde ... Together Again
rating: 

Καταλήγουμε στο ίδιο αποτέλεσμα με την σταδιακή εκπαίδευση του μοντέλου μας.


### Αποτελέσματα


Καθώς το precision είναι περίπου 50% ,ενώ το recall 0,66 βλέπουμε πως το μοντέλο προτείνει ταινίες εκ των οποίων περισσότερες από τις μισές προτείνονται ορθώς.Σίγουρα το γεγονός πως το recall ειναι 0.66 σημαίνει πως υπάρχουν περιθώρια για βελτίωση, καθώς γίνεται missclasify 1 στις 3 ταινίες.Δηλαδή ταξινομούνται ως μη-προτεινόμενες ενώ θα έπρεπε να συμβαίνει το ανάποδο.Το γεγονός όμως πως έχουμε δεδομένα μονάχα από 1 χρήστη μας οδηγεί στο συμπέρασμα πως δεν έχει νόημα η συστηματική προσπάθεια να επιτύχουμε recall 100%,διότι το μοντέλο σε αυτή την περίπτωση πολύ πιθανόν να παρουσιάζει φαινόμενα overfitiing.Αν'αυτού, προσπαθήσαμε να καθορίσουμε κάποιες γενικές αρχές για το σύστημα συστάσεων που (θεωρητικά) θα μπορούσαν να ισχύουν για μια πληθώρα ανθρώπων.Δυστυχώς στα πλαίσια αυτής της εργασίας δεν είναι δυνατό να εξακριβώσουμε την ικανότητα γενίκευσης του μοντέλου μας.